In [8]:
import sqlalchemy
import pandas as pd
import os
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import inspect
import openpyxl
%reload_ext autoreload
%autoreload 2
from geolocate import geolocate_batch
from utils import *

In [9]:
server = 'localhost'
database = 't_obras'
username = 'root'
password = input("Digite a senha do mysql: ")
driver = 'mysql+pymysql'
connection_string = f'{driver}://{username}:{password}@{server}/{database}'
engine = sqlalchemy.create_engine(connection_string)

In [10]:
local = pd.read_sql_table('localizacoes_obra', con=engine)
local.head()

,CD_OBRA,ENDERECO,NM_BAIRRO,NM_RA,NM_AP
0,1,None,None,None,AP 1.1
1,1,None,None,None,AP 2.2
2,1,None,None,None,AP 3.1
3,1,None,None,None,AP 4.1
4,2,None,SANTA TERESA,Santa Teresa,AP 1.1


In [11]:
# criar coluna ENDERECO + BAIRRO + 'RIO DE JANEIRO, BRASIL'
local['endereco_completo'] = local['ENDERECO'] + ', ' + local['NM_BAIRRO'] + ', RIO DE JANEIRO, BRASIL'
local.head(5)

,CD_OBRA,ENDERECO,NM_BAIRRO,NM_RA,NM_AP,endereco_completo
0,1,None,None,None,AP 1.1,NaN
1,1,None,None,None,AP 2.2,NaN
2,1,None,None,None,AP 3.1,NaN
3,1,None,None,None,AP 4.1,NaN
4,2,None,SANTA TERESA,Santa Teresa,AP 1.1,NaN


In [12]:
# limpar coluna endereco completo strip
local['endereco_completo'] = local['endereco_completo'].astype(str).str.strip()
# limpar caracteres especiais
local['endereco_completo'] = local['endereco_completo'].astype(str).str.replace('[^A-Za-z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ/ºª ]', '')

<ipython-input-12-5ed00fd980c2>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  local['endereco_completo'] = local['endereco_completo'].astype(str).str.replace('[^A-Za-z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ/ºª ]', '')


In [13]:
aps = pd.read_sql_table('coordenadas_aps', engine)

In [15]:
aps.head()

,AP,Bairro,Latitude/Longitude,Endereco,NM_AP
0,1.1,Centro,"-22.907090, -43.208930","R. Pedro Alves, 311 - Santo Cristo, Rio de Jan...",AP 1.1
1,2.1,Zona Sul,"-22.968931, -43.201832","Lagoa, Rio de Janeiro - RJ",AP 2.1
2,2.2,Tijuca,"-22.943738, -43.240806","R. Castelo Novo, 413 - Tijuca, Rio de Janeiro ...",AP 2.2
3,3.1,Ramos,"-22.858152, -43.247817","Rua 17 de Fevereiro, 102C - Bonsucesso, Rio de...",AP 3.1
4,3.2,Meier,"-22.899457, -43.280302","Av. Amaro Cavalcanti, 15 - Méier, Rio de Janei...",AP 3.2


In [18]:
len(local)

4466

In [20]:
# onde endereco_completo for Nan ou vazio substituir com o endereço da aps
for index, row in local.iterrows():
    if row['endereco_completo'] == 'nan' or row['endereco_completo'] == '':
        local.loc[index, 'endereco_completo'] = aps.loc[aps['NM_AP'] == row['NM_AP'], 'Endereco'].values[0]
    else:
        pass

In [21]:
local.head(50)

,CD_OBRA,ENDERECO,NM_BAIRRO,NM_RA,NM_AP,endereco_completo
0,1,None,None,None,AP 1.1,"R. Pedro Alves, 311 - Santo Cristo, Rio de Jan..."
1,1,None,None,None,AP 2.2,"R. Castelo Novo, 413 - Tijuca, Rio de Janeiro ..."
2,1,None,None,None,AP 3.1,"Rua 17 de Fevereiro, 102C - Bonsucesso, Rio de..."
3,1,None,None,None,AP 4.1,"Salao Beleza Da Mulher, R. Min. Gabriel de Piz..."
4,2,None,SANTA TERESA,Santa Teresa,AP 1.1,"R. Pedro Alves, 311 - Santo Cristo, Rio de Jan..."
5,3,RUA DOUTOR JOVINIANO,MADUREIRA,Madureira,AP 3.3,RUA DOUTOR JOVINIANO MADUREIRA RIO DE JANEIRO ...
6,4,RUA MANUEL MACHADO,VAZ LOBO,Madureira,AP 3.3,RUA MANUEL MACHADO VAZ LOBO RIO DE JANEIRO BRASIL
7,5,ESTRADA PAULO DE MEDEIROS,AGUA SANTA,Méier,AP 3.1,ESTRADA PAULO DE MEDEIROS AGUA SANTA RIO DE JA...
8,6,RUA MILETO MACIEL,CACUIA,Ilha do Governador,AP 3.2,RUA MILETO MACIEL CACUIA RIO DE JANEIRO BRASIL
9,7,ESTRADA ROBERTO BURLE MARX,BARRA DE GUARATIBA,Guaratiba,AP 5.3,ESTRADA ROBERTO BURLE MARX BARRA DE GUARATIBA ...


In [22]:
local.columns

Index(['CD_OBRA', 'ENDERECO', 'NM_BAIRRO', 'NM_RA', 'NM_AP',
       'endereco_completo'],
      dtype='object')

In [25]:
end = local['endereco_completo'].astype(str).str.strip()

In [26]:
end_geo = geolocate_batch(end)

100%|██████████| 4466/4466 [02:24<00:00, 30.89it/s]


In [27]:
local['lat'] = [lat['latitude'] for lat in end_geo]
local['long'] = [lat['longitude'] for lat in end_geo]

In [28]:
local.head()

,CD_OBRA,ENDERECO,NM_BAIRRO,NM_RA,NM_AP,endereco_completo,lat,long
0,1,None,None,None,AP 1.1,"R. Pedro Alves, 311 - Santo Cristo, Rio de Jan...",-22.904503,-43.206717
1,1,None,None,None,AP 2.2,"R. Castelo Novo, 413 - Tijuca, Rio de Janeiro ...",-22.942466,-43.242727
2,1,None,None,None,AP 3.1,"Rua 17 de Fevereiro, 102C - Bonsucesso, Rio de...",-22.858405,-43.247792
3,1,None,None,None,AP 4.1,"Salao Beleza Da Mulher, R. Min. Gabriel de Piz...",-22.941588,-43.360832
4,2,None,SANTA TERESA,Santa Teresa,AP 1.1,"R. Pedro Alves, 311 - Santo Cristo, Rio de Jan...",-22.904503,-43.206717


In [29]:
local.head(100)

,CD_OBRA,ENDERECO,NM_BAIRRO,NM_RA,NM_AP,endereco_completo,lat,long
0,1,None,None,None,AP 1.1,"R. Pedro Alves, 311 - Santo Cristo, Rio de Jan...",-22.904503,-43.206717
1,1,None,None,None,AP 2.2,"R. Castelo Novo, 413 - Tijuca, Rio de Janeiro ...",-22.942466,-43.242727
2,1,None,None,None,AP 3.1,"Rua 17 de Fevereiro, 102C - Bonsucesso, Rio de...",-22.858405,-43.247792
3,1,None,None,None,AP 4.1,"Salao Beleza Da Mulher, R. Min. Gabriel de Piz...",-22.941588,-43.360832
4,2,None,SANTA TERESA,Santa Teresa,AP 1.1,"R. Pedro Alves, 311 - Santo Cristo, Rio de Jan...",-22.904503,-43.206717
...,...,...,...,...,...,...,...,...
95,168,None,None,None,AP 4.1,"Salao Beleza Da Mulher, R. Min. Gabriel de Piz...",-22.941588,-43.360832
96,168,None,None,None,AP 5.1,"Tv. da Prata, 37-1 - Bangu, Rio de Janeiro - R...",-22.868778,-43.468656
97,168,None,None,None,AP 5.2,"R. Vítor Alves, 1429 - Campo Grande, Rio de Ja...",-22.890096,-43.566908
98,168,None,None,None,AP 5.3,"R. dos Bambus, 162 - Santa Cruz, Rio de Janeir...",-22.910295,-43.679601


In [30]:
# salvar no banco de dados
local.to_sql('localizacoes_obra_ap', engine, if_exists='replace', index=False)